In [1]:
## Import the libraries
import os
import numpy as np
from PIL import Image
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Flatten, BatchNormalization # Flatten will be used to applied between convolution layer and MLP(matrix to vector)
from keras.layers import Convolution2D, MaxPooling2D , ZeroPadding2D # Convolution 2D for images, for text use convolution1D
from keras.utils import to_categorical 
from sklearn.preprocessing import LabelEncoder
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
import tensorflow as tf
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib

In [3]:
path = r"C:\Users\ashwa\Desktop\Image_Classification\All_61326\train/"

#Extracted list of all folders 
dirs = os.listdir(path)

MODEL_NAME = 'Image_Convnet'

## Batch size
batch_size = 32

# Total number of classes(images)
nb_classes = 6
nb_epoch = 50

img_rows, img_cols = 150, 150     # input image dimensions
img_channels = 3                  # images are RGB. as colored images

In [4]:
# function which resize images and stores in list(lst) and stores all train labels in list(target)
def resize(lst,target,path,dirs):
    print("Resizing the images!!")
    for folder in dirs:
        for img in os.listdir(path+folder):
            if os.path.isfile(path+folder+"/"+img):
                im = Image.open(path+folder+"/"+img)
                imResize = np.array(im.resize((150,150),Image.ANTIALIAS))
                lst.append(imResize)
                target.append(folder)
    return(lst,target)

In [5]:
# convert all elements of the list to array
def lst_to_array(lst):
    print("Converting to Numpy array!!")
    for i in range(0,len(lst)):
        lst[i] = np.asarray(lst[i])
        return lst

In [6]:
def label_encoder(target):
    print("Label Encoder!!")
    # converts the character array to numeric array. Assigns levels to unique labels.
    le = LabelEncoder()                  
    le.fit(target)
    train_labels = le.transform(target)
    #val_labels = le.transform(val_target)# convert string into numbers
    
    print(le.classes_)
    
    # find unique train labels
    print(np.unique(target, return_counts=True))    
    print(len(train_labels))
    
    # Converted train and validation labels to one hot encoding
    labels_train = to_categorical(np.asarray(train_labels))

    return labels_train

In [7]:
def cnn_model(lst, labels_train):
    print("Fit the Model!!")
    
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64))
    model.add(Dropout(0.4))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(64))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(Dropout(0.7))
    model.add(Dense(6, activation = "softmax"))
    
    model.summary()
    
    ## Compile the model
    #Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    adam = Adam(lr = 0.001,decay= 0.0001)
    model.compile(loss='categorical_crossentropy', # Becoz it is multi class classification problem
              optimizer= adam,
              metrics=['accuracy'])
    
    ## Fit the model
    model.fit(np.array(lst),labels_train,
          batch_size=batch_size,
          epochs=20,
          validation_split=0.1,
          shuffle=True)
    
    return model

In [8]:
def main():
    
    # list to store all images of train data 
    lst = []
    # list to store all labels of train data
    target = []

    ## calling function to extract train image data and their label in list
    lst, target = resize(lst, target, path, dirs)

    ## Converting the list of images to an array
    lst = lst_to_array(lst)
    
    ## Encoding the labels
    labels_train = label_encoder(target)
    
    ## Build a CNN Model
    cnn_model(lst, labels_train)

In [9]:
if __name__ == '__main__':
    main()

Resizing the images!!
Converting to Numpy array!!
Label Encoder!!
['61326 Scratch Mark' '61326 Slot Damage' '61326 Thinning' '61326 Wrinkle'
 '61326_0k_back' '61326_ok_front']
(array(['61326 Scratch Mark', '61326 Slot Damage', '61326 Thinning',
       '61326 Wrinkle', '61326_0k_back', '61326_ok_front'],
      dtype='<U18'), array([385, 330, 488, 390, 415, 406], dtype=int64))
2414
Fit the Model!!
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 64)        18496     
_______

In [10]:

## Reading the Test Images from the repective directories
test_datagen = ImageDataGenerator()

test_generator = test_datagen.flow_from_directory(
    'test_61326/',  # this is the target directory
    target_size=(150, 150),  # all images will be resized to 150x150
    class_mode='categorical')

Found 6 images belonging to 6 classes.


In [11]:
## Predictions for the test files
test_prob = model.predict_generator(test_generator) # this returns the probabilities
test_pred_classes = np.argmax(test_prob, axis=1) # convert probabilities to classes
print(test_pred_classes)
print(test_prob)

NameError: name 'model' is not defined

In [ ]:
## Creating checkpoints and .pb file

def export_model(saver, model, input_node_names, output_node_name):
    tf.train.write_graph(K.get_session().graph_def, 'out', \
        MODEL_NAME + '_graph.pbtxt')

    saver.save(K.get_session(), 'out/' + MODEL_NAME + '.chkp')

    freeze_graph.freeze_graph('out/' + MODEL_NAME + '_graph.pbtxt', None, \
        False, 'out/' + MODEL_NAME + '.chkp', output_node_name, \
        "save/restore_all", "save/Const:0", \
        'out/frozen_' + MODEL_NAME + '.pb', True, "")

    input_graph_def = tf.GraphDef()
    with tf.gfile.Open('out/frozen_' + MODEL_NAME + '.pb', "rb") as f:
        input_graph_def.ParseFromString(f.read())

    output_graph_def = optimize_for_inference_lib.optimize_for_inference(
            input_graph_def, input_node_names, [output_node_name],
            tf.float32.as_datatype_enum)

    with tf.gfile.FastGFile('out/opt_' + MODEL_NAME + '.pb', "wb") as f:
        f.write(output_graph_def.SerializeToString())

    print("graph saved!")


In [ ]:
export_model(tf.train.Saver(), model, ["conv2d_1_input"], "0")